In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import ast
import json
import time
from datetime import datetime

In [2]:
movies_original = pd.read_csv("movies_dataset.csv")
movies_original.head()

/tmp/ipykernel_11909/352882099.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_original = pd.read_csv("movies_dataset.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Lo primero será desanidar los datos en las columnas anidadas, iniciando por:
### belongs_to_collection

Esto se consigue usando la librería `ast`, la cual nos permite analizar y trabajar con árboles, lo que en este proyecto se traduce a poder manejar más fácilmente los diccionarios que estaban en un inicio en formato `string`

In [ ]:
movies_copy = movies_original.copy()

# toma los valores que están inicialmente como strings y los transforma en diccionarios
series_from_col = movies_copy["belongs_to_collection"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)
series_from_col.head(5)

0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                  NaN
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                  NaN
4    {'id': 96871, 'name': 'Father of the Bride Col...
Name: belongs_to_collection, dtype: object

In [ ]:
unnested_dataframe = pd.json_normalize(series_from_col)
result_dataframe = pd.concat([movies_copy.drop(columns=['belongs_to_collection']), unnested_dataframe], axis=1)
result_dataframe.head(5)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,status,tagline,title,video,vote_average,vote_count,id,name,poster_path,backdrop_path
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,Released,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN,NaN,NaN,NaN
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


### genres

Esta columna también debe desanidarse

In [ ]:
dataframe_from_col2 = pd.DataFrame(columns=['genre_id','genre_name'])

for value in result_dataframe["genres"]:
  data = ast.literal_eval(value)
  ids = [item['id'] for item in data]
  names = [item['name'] for item in data]
  dataframe_from_col2.loc[len(dataframe_from_col2)] = [ids,names]

dataframe_from_col2.head(5)

,genre_id,genre_name
0,"[16, 35, 10751]","[Animation, Comedy, Family]"
1,"[12, 14, 10751]","[Adventure, Fantasy, Family]"
2,"[10749, 35]","[Romance, Comedy]"
3,"[35, 18, 10749]","[Comedy, Drama, Romance]"
4,[35],[Comedy]


In [ ]:
result_dataframe = result_dataframe.join(dataframe_from_col2)
result_dataframe.head(5)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,title,video,vote_average,vote_count,id,name,poster_path,backdrop_path,genre_id,genre_name
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[16, 35, 10751]","[Animation, Comedy, Family]"
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN,"[12, 14, 10751]","[Adventure, Fantasy, Family]"
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,Grumpier Old Men,False,6.5,92.0,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[10749, 35]","[Romance, Comedy]"
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,Waiting to Exhale,False,6.1,34.0,NaN,NaN,NaN,NaN,"[35, 18, 10749]","[Comedy, Drama, Romance]"
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,Father of the Bride Part II,False,5.7,173.0,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,[35],[Comedy]


### production_companies

Esta es la siguiente columna a desanidar. Se procederá de igual forma que con la columna genres. Aunque lo ideal sería crear una función, aún no comprendo bien el funcionamiento de la librerías `ast` por lo que debo evaluarla para cada columna

In [ ]:
dataframe_from_col3 = pd.DataFrame(columns=['production_company_name','production_company_id'])

for value in result_dataframe["production_companies"]:
    if pd.notna(value):  # Check if the value is not NaN
        data = ast.literal_eval(value)
        if isinstance(data, list) and len(data) > 0:
            ids = [item['id'] for item in data]
            names = [item['name'] for item in data]
            dataframe_from_col3.loc[len(dataframe_from_col3)] = [names, ids]
        else:
            dataframe_from_col3.loc[len(dataframe_from_col3)] = [None, None]
    else:
        dataframe_from_col3.loc[len(dataframe_from_col3)] = [None, None]

dataframe_from_col3.head(5)

,production_company_name,production_company_id
0,[Pixar Animation Studios],[3]
1,"[TriStar Pictures, Teitler Film, Interscope Co...","[559, 2550, 10201]"
2,"[Warner Bros., Lancaster Gate]","[6194, 19464]"
3,[Twentieth Century Fox Film Corporation],[306]
4,"[Sandollar Productions, Touchstone Pictures]","[5842, 9195]"


In [ ]:
result_dataframe=result_dataframe.join(dataframe_from_col3)
result_dataframe.head(5)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,vote_average,vote_count,id,name,poster_path,backdrop_path,genre_id,genre_name,production_company_name,production_company_id
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[16, 35, 10751]","[Animation, Comedy, Family]",[Pixar Animation Studios],[3]
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,6.9,2413.0,NaN,NaN,NaN,NaN,"[12, 14, 10751]","[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...","[559, 2550, 10201]"
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,6.5,92.0,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[10749, 35]","[Romance, Comedy]","[Warner Bros., Lancaster Gate]","[6194, 19464]"
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,6.1,34.0,NaN,NaN,NaN,NaN,"[35, 18, 10749]","[Comedy, Drama, Romance]",[Twentieth Century Fox Film Corporation],[306]
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,5.7,173.0,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,[35],[Comedy],"[Sandollar Productions, Touchstone Pictures]","[5842, 9195]"


### production_countries

Lo mismo aquí tal cual como para `production_company`

In [ ]:
dataframe_from_col4 = pd.DataFrame(columns=['production_iso_3166_1','production_country'])

for value in result_dataframe["production_countries"]:
    if pd.notna(value):  # Check if the value is not NaN
        data = ast.literal_eval(value)
        if isinstance(data, list) and len(data) > 0:
            iso_3166_1 = [item['iso_3166_1'] for item in data]
            names = [item['name'] for item in data]
            dataframe_from_col4.loc[len(dataframe_from_col4)] = [iso_3166_1[0], names[0]]
        else:
            dataframe_from_col4.loc[len(dataframe_from_col4)] = [None, None]
    else:
        dataframe_from_col4.loc[len(dataframe_from_col4)] = [None, None]

dataframe_from_col4.head(5)

,production_iso_3166_1,production_country
0,US,United States of America
1,US,United States of America
2,US,United States of America
3,US,United States of America
4,US,United States of America


In [ ]:
result_dataframe=result_dataframe.join(dataframe_from_col4)
result_dataframe.head(5)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,id,name,poster_path,backdrop_path,genre_id,genre_name,production_company_name,production_company_id,production_iso_3166_1,production_country
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[16, 35, 10751]","[Animation, Comedy, Family]",[Pixar Animation Studios],[3],US,United States of America
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,NaN,NaN,NaN,NaN,"[12, 14, 10751]","[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...","[559, 2550, 10201]",US,United States of America
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[10749, 35]","[Romance, Comedy]","[Warner Bros., Lancaster Gate]","[6194, 19464]",US,United States of America
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,NaN,NaN,NaN,NaN,"[35, 18, 10749]","[Comedy, Drama, Romance]",[Twentieth Century Fox Film Corporation],[306],US,United States of America
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,[35],[Comedy],"[Sandollar Productions, Touchstone Pictures]","[5842, 9195]",US,United States of America


### spoken_languages

Última columna por desanidar

In [ ]:
dataframe_from_col5 = pd.DataFrame(columns=['language_iso_639_1','language'])

for value in result_dataframe["spoken_languages"]:
    if pd.notna(value):  # Check if the value is not NaN
        data = ast.literal_eval(value)
        if isinstance(data, list) and len(data) > 0:
            iso_639_1 = [item['iso_639_1'] for item in data]
            language = [item['name'] for item in data]
            dataframe_from_col5.loc[len(dataframe_from_col5)] = [iso_639_1[0], language[0]]
        else:
            dataframe_from_col5.loc[len(dataframe_from_col5)] = [None, None]
    else:
        dataframe_from_col5.loc[len(dataframe_from_col5)] = [None, None]

dataframe_from_col5.head(5)

,language_iso_639_1,language
0,en,English
1,en,English
2,en,English
3,en,English
4,en,English


In [ ]:
result_dataframe = result_dataframe.join(dataframe_from_col5)
result_dataframe.head(5)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,poster_path,backdrop_path,genre_id,genre_name,production_company_name,production_company_id,production_iso_3166_1,production_country,language_iso_639_1,language
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[16, 35, 10751]","[Animation, Comedy, Family]",[Pixar Animation Studios],[3],US,United States of America,en,English
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,NaN,NaN,"[12, 14, 10751]","[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...","[559, 2550, 10201]",US,United States of America,en,English
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[10749, 35]","[Romance, Comedy]","[Warner Bros., Lancaster Gate]","[6194, 19464]",US,United States of America,en,English
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,NaN,NaN,"[35, 18, 10749]","[Comedy, Drama, Romance]",[Twentieth Century Fox Film Corporation],[306],US,United States of America,en,English
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,[35],[Comedy],"[Sandollar Productions, Touchstone Pictures]","[5842, 9195]",US,United States of America,en,English


In [ ]:
len(result_dataframe.columns)

35

In [ ]:
result_dataframe.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'id', 'name', 'poster_path',
       'backdrop_path', 'genre_id', 'genre_name', 'production_company_name',
       'production_company_id', 'production_iso_3166_1', 'production_country',
       'language_iso_639_1', 'language'],
      dtype='object')

Ahora con las columnas desanidadas, se procede a eliminar las que contienen los datos anidados del dataframe y luego se guarda un nuevo archivo csv

In [ ]:
result_dataframe.drop(columns=['genres','production_companies','production_countries','spoken_languages'],axis=1, inplace=True)
len(result_dataframe.columns)

31

In [ ]:
result_dataframe.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'id', 'name', 'poster_path',
       'backdrop_path', 'genre_id', 'genre_name', 'production_company_name',
       'production_company_id', 'production_iso_3166_1', 'production_country',
       'language_iso_639_1', 'language'],
      dtype='object')

### Filling and dropping null or NaN values

Ahora se manejan los datos faltantes

In [ ]:
result_dataframe.fillna({"revenue":0,"budget":0},inplace=True)

In [ ]:
result_dataframe.isna().sum()

adult                          0
budget                         0
homepage                   37684
id                             0
imdb_id                       17
original_language             11
original_title                 0
overview                     954
popularity                     5
poster_path                  386
release_date                  87
revenue                        0
runtime                      263
status                        87
tagline                    25054
title                          6
video                          6
vote_average                   6
vote_count                     6
id                         40975
name                       40975
poster_path                41518
backdrop_path              42203
genre_id                       0
genre_name                     0
production_company_name    11881
production_company_id      11881
production_iso_3166_1       6288
production_country          6288
language_iso_639_1          3835
language  

In [ ]:
result_dataframe.dropna(subset=["release_date"], inplace=True)

In [ ]:
result_dataframe.isna().sum()

adult                          0
budget                         0
homepage                   37610
id                             0
imdb_id                       14
original_language             11
original_title                 0
overview                     941
popularity                     2
poster_path                  339
release_date                   0
revenue                        0
runtime                      249
status                        83
tagline                    24981
title                          3
video                          3
vote_average                   3
vote_count                     3
id                         40891
name                       40891
poster_path                41434
backdrop_path              42119
genre_id                       0
genre_name                     0
production_company_name    11799
production_company_id      11799
production_iso_3166_1       6214
production_country          6214
language_iso_639_1          3771
language  

In [ ]:
result_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45379 entries, 0 to 45465
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   adult                    45379 non-null  object 
 1   budget                   45379 non-null  object 
 2   homepage                 7769 non-null   object 
 3   id                       45379 non-null  object 
 4   imdb_id                  45365 non-null  object 
 5   original_language        45368 non-null  object 
 6   original_title           45379 non-null  object 
 7   overview                 44438 non-null  object 
 8   popularity               45377 non-null  object 
 9   poster_path              45040 non-null  object 
 10  release_date             45379 non-null  object 
 11  revenue                  45379 non-null  float64
 12  runtime                  45130 non-null  float64
 13  status                   45296 non-null  object 
 14  tagline                  20

In [ ]:
data_copy = result_dataframe.copy()

In [ ]:
data_copy['release_date'] = pd.to_datetime(data_copy['release_date'], errors='coerce')

In [ ]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45379 entries, 0 to 45465
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   adult                    45379 non-null  object        
 1   budget                   45379 non-null  object        
 2   homepage                 7769 non-null   object        
 3   id                       45379 non-null  object        
 4   imdb_id                  45365 non-null  object        
 5   original_language        45368 non-null  object        
 6   original_title           45379 non-null  object        
 7   overview                 44438 non-null  object        
 8   popularity               45377 non-null  object        
 9   poster_path              45040 non-null  object        
 10  release_date             45376 non-null  datetime64[ns]
 11  revenue                  45379 non-null  float64       
 12  runtime                  45130 non-nu

In [ ]:
data_copy["release_year"] = data_copy["release_date"].dt.year
data_copy.dropna(subset=["release_year"],inplace=True)

In [ ]:
data_copy["release_year"] = data_copy["release_year"].astype(int)
data_copy

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,backdrop_path,genre_id,genre_name,production_company_name,production_company_id,production_iso_3166_1,production_country,language_iso_639_1,language,release_year
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[16, 35, 10751]","[Animation, Comedy, Family]",[Pixar Animation Studios],[3],US,United States of America,en,English,1995
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,NaN,"[12, 14, 10751]","[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...","[559, 2550, 10201]",US,United States of America,en,English,1995
2,False,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[10749, 35]","[Romance, Comedy]","[Warner Bros., Lancaster Gate]","[6194, 19464]",US,United States of America,en,English,1995
3,False,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,NaN,"[35, 18, 10749]","[Comedy, Drama, Romance]",[Twentieth Century Fox Film Corporation],[306],US,United States of America,en,English,1995
4,False,0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,[35],[Comedy],"[Sandollar Productions, Touchstone Pictures]","[5842, 9195]",US,United States of America,en,English,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,False,0,NaN,30840,tt0102797,en,Robin Hood,"Yet another version of the classic epic, with ...",5.683753,/fQC46NglNiEMZBv5XHoyLuOWoN5.jpg,...,NaN,"[18, 28, 10749]","[Drama, Action, Romance]","[Westdeutscher Rundfunk (WDR), Working Title F...","[7025, 10163, 16323, 38978]",CA,Canada,en,English,1991
45462,False,0,NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,...,NaN,[18],[Drama],[Sine Olivia],[19653],PH,Philippines,tl,,2011
45463,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,NaN,"[28, 18, 53]","[Action, Drama, Thriller]",[American World Pictures],[6165],US,United States of America,en,English,2003
45464,False,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,NaN,[],[],[Yermoliev],[88753],RU,Russia,None,None,1917


In [5]:
unnested_data = pd.read_csv("movies_dataset_unnested.csv")
unnested_data.tail(5)

/tmp/ipykernel_11909/3206048439.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  unnested_data = pd.read_csv("movies_dataset_unnested.csv")


,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,poster_path.1,backdrop_path,genre_id,genre_name,production_company_name,production_company_id,production_iso_3166_1,production_country,language_iso_639_1,language
45461,False,0,http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,...,NaN,NaN,"[18, 10751]","['Drama', 'Family']",NaN,NaN,IR,Iran,fa,فارسی
45462,False,0,NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,...,NaN,NaN,[18],['Drama'],['Sine Olivia'],[19653],PH,Philippines,tl,NaN
45463,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,NaN,NaN,"[28, 18, 53]","['Action', 'Drama', 'Thriller']",['American World Pictures'],[6165],US,United States of America,en,English
45464,False,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,NaN,NaN,[],[],['Yermoliev'],[88753],RU,Russia,NaN,NaN
45465,False,0,NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,0.163015,/s5UkZt6NTsrS7ZF0Rh8nzupRlIU.jpg,...,NaN,NaN,[],[],NaN,NaN,GB,United Kingdom,en,English


In [10]:
non_numeric = pd.to_numeric(unnested_data["budget"],errors='coerce')
unnested_data[non_numeric.isna()]

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,poster_path.1,backdrop_path,genre_id,genre_name,production_company_name,production_company_id,production_iso_3166_1,production_country,language_iso_639_1,language
19730,- Written by Ørnås,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,...,NaN,NaN,"[11176, 11602, 29812]","['Carousel Productions', 'Vision View Entertai...",NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,...,NaN,NaN,"[2883, 7759, 7760, 7761, 33751]","['Aniplex', 'GoHands', 'BROSTA TV', 'Mardock S...",NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,...,NaN,NaN,"[17161, 18012, 18013, 23822]","['Odyssey Media', 'Pulser Productions', 'Rogue...",NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
unnested_data_without_nan = unnested_data[non_numeric.notna()]
unnested_data_without_nan

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,poster_path.1,backdrop_path,genre_id,genre_name,production_company_name,production_company_id,production_iso_3166_1,production_country,language_iso_639_1,language
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[16, 35, 10751]","['Animation', 'Comedy', 'Family']",['Pixar Animation Studios'],[3],US,United States of America,en,English
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,NaN,NaN,"[12, 14, 10751]","['Adventure', 'Fantasy', 'Family']","['TriStar Pictures', 'Teitler Film', 'Intersco...","[559, 2550, 10201]",US,United States of America,en,English
2,False,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[10749, 35]","['Romance', 'Comedy']","['Warner Bros.', 'Lancaster Gate']","[6194, 19464]",US,United States of America,en,English
3,False,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,NaN,NaN,"[35, 18, 10749]","['Comedy', 'Drama', 'Romance']",['Twentieth Century Fox Film Corporation'],[306],US,United States of America,en,English
4,False,0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,[35],['Comedy'],"['Sandollar Productions', 'Touchstone Pictures']","[5842, 9195]",US,United States of America,en,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,0,http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,...,NaN,NaN,"[18, 10751]","['Drama', 'Family']",NaN,NaN,IR,Iran,fa,فارسی
45462,False,0,NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,...,NaN,NaN,[18],['Drama'],['Sine Olivia'],[19653],PH,Philippines,tl,NaN
45463,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,NaN,NaN,"[28, 18, 53]","['Action', 'Drama', 'Thriller']",['American World Pictures'],[6165],US,United States of America,en,English
45464,False,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,NaN,NaN,[],[],['Yermoliev'],[88753],RU,Russia,NaN,NaN


In [14]:
unnested_data_without_nan["budget"] = unnested_data_without_nan["budget"].astype(int)
unnested_data_without_nan["budget"]

/tmp/ipykernel_11909/3253403878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unnested_data_without_nan["budget"] = unnested_data_without_nan["budget"].astype(int)


0        30000000
1        65000000
2               0
3        16000000
4               0
           ...   
45461           0
45462           0
45463           0
45464           0
45465           0
Name: budget, Length: 45463, dtype: int64

In [19]:
unnested_data_without_nan["return"] = unnested_data_without_nan["revenue"].div(unnested_data_without_nan["budget"]).replace(np.nan,0).replace(np.inf,0)
unnested_data_without_nan["return"]

/tmp/ipykernel_11909/1090359219.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unnested_data_without_nan["return"] = unnested_data_without_nan["revenue"].div(unnested_data_without_nan["budget"]).replace(np.nan,0).replace(np.inf,0)


0        12.451801
1         4.043035
2         0.000000
3         5.090760
4         0.000000
           ...    
45461     0.000000
45462     0.000000
45463     0.000000
45464     0.000000
45465     0.000000
Name: return, Length: 45463, dtype: float64

In [20]:
final_movies_dataset = unnested_data_without_nan.copy()
final_movies_dataset.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'id.1', 'name', 'poster_path.1',
       'backdrop_path', 'genre_id', 'genre_name', 'production_company_name',
       'production_company_id', 'production_iso_3166_1', 'production_country',
       'language_iso_639_1', 'language', 'return'],
      dtype='object')

In [21]:
final_movies_dataset.drop(columns=['video','imdb_id','adult','original_title','poster_path','homepage'],axis=1,inplace=True)
final_movies_dataset.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'id.1', 'name', 'poster_path.1',
       'backdrop_path', 'genre_id', 'genre_name', 'production_company_name',
       'production_company_id', 'production_iso_3166_1', 'production_country',
       'language_iso_639_1', 'language', 'return'],
      dtype='object')

In [22]:
final_movies_dataset.to_csv("movies_dataset_final.csv",index=False)

In [2]:
# remove duplicated

df = pd.read_csv("movies_dataset_final.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   budget                   45463 non-null  int64  
 1   id                       45463 non-null  int64  
 2   original_language        45452 non-null  object 
 3   overview                 44509 non-null  object 
 4   popularity               45460 non-null  float64
 5   release_date             45376 non-null  object 
 6   revenue                  45460 non-null  float64
 7   runtime                  45203 non-null  float64
 8   status                   45379 non-null  object 
 9   tagline                  20412 non-null  object 
 10  title                    45460 non-null  object 
 11  vote_average             45460 non-null  float64
 12  vote_count               45460 non-null  float64
 13  id.1                     4491 non-null   float64
 14  name                  

In [3]:
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45446 entries, 0 to 45462
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   budget                   45446 non-null  int64  
 1   id                       45446 non-null  int64  
 2   original_language        45435 non-null  object 
 3   overview                 44492 non-null  object 
 4   popularity               45443 non-null  float64
 5   release_date             45359 non-null  object 
 6   revenue                  45443 non-null  float64
 7   runtime                  45186 non-null  float64
 8   status                   45362 non-null  object 
 9   tagline                  20407 non-null  object 
 10  title                    45443 non-null  object 
 11  vote_average             45443 non-null  float64
 12  vote_count               45443 non-null  float64
 13  id.1                     4490 non-null   float64
 14  name                     44

In [4]:
df.to_csv("movies_dataset_cleaned.csv",index=False)